In [1]:
import os
from tensorflow import keras
from keras.preprocessing import image
import random
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense

In [11]:
categories=['with_mask','without_mask']

In [12]:
for category in categories:
    path=os.path.join('train',category)
    print(path)

train\with_mask
train\without_mask


In [50]:
data=[]
train='C:\\Users\\APOORVA\\Desktop\\MACHINE LEARNING\\PROJECTS\\Face Mask Detection\\face-mask-dataset\\Dataset\\train\\train'
for category in categories:
    path=os.path.join(train,category)
    print(path)
    label=categories.index(category)
    
    for file in os.listdir(path):
        
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        
        data.append([img,label])
        

C:\Users\APOORVA\Desktop\MACHINE LEARNING\PROJECTS\Face Mask Detection\face-mask-dataset\Dataset\train\train\with_mask
C:\Users\APOORVA\Desktop\MACHINE LEARNING\PROJECTS\Face Mask Detection\face-mask-dataset\Dataset\train\train\without_mask


In [14]:
len(data)

1315

In [15]:
# To avoid bias
random.shuffle(data)

In [16]:
# Converting X and Y in data to numpy array

X=[]
y=[]

for features, label in data:
    X.append(features)
    y.append(label)    

In [17]:
len(X)

1315

In [18]:
len(y)

1315

In [19]:
X=np.array(X)
y=np.array(y)

In [20]:
# X is 3D
X.shape

(1315, 224, 224, 3)

In [21]:
y.shape

(1315,)

In [22]:
# Standardize X
X=X/255

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [24]:
X_train.shape

(1052, 224, 224, 3)

In [25]:
X_test.shape

(263, 224, 224, 3)

In [26]:
vgg=VGG16()

In [27]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [28]:
model=Sequential()

In [29]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

Now we will freeze all the layers so on training, weights are not updated

In [31]:
for layer in model.layers:
    layer.trainable=False

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [33]:
model.add(Dense(1,activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [35]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [36]:
model.fit(X_train,y_train,epochs=5,batch_size=2,validation_data=(X_test,y_test))

Epoch 1/5
526/526 [==============================] - 383s 705ms/step - loss: 0.3491 - accuracy: 0.8517 - val_loss: 0.1815 - val_accuracy: 0.9468
Epoch 2/5
526/526 [==============================] - 367s 698ms/step - loss: 0.1654 - accuracy: 0.9411 - val_loss: 0.1266 - val_accuracy: 0.9658
Epoch 3/5
526/526 [==============================] - 357s 679ms/step - loss: 0.1408 - accuracy: 0.9420 - val_loss: 0.0966 - val_accuracy: 0.9810
Epoch 4/5
526/526 [==============================] - 321s 611ms/step - loss: 0.0907 - accuracy: 0.9715 - val_loss: 0.1667 - val_accuracy: 0.9240
Epoch 5/5
526/526 [==============================] - 297s 566ms/step - loss: 0.0785 - accuracy: 0.9715 - val_loss: 0.0693 - val_accuracy: 0.9848


In [37]:
cap=cv2.VideoCapture(0)

In [48]:
while True:
    
    ret,frame=cap.read()
    
    # call the detection method
    
    img=cv2.resize(frame,(224,224))

    y_pred=detect_face_mask(img)
    
    if y_pred == 0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

ret=True or False values

frame=image

img.reshape(1,224,224,3)

here 1 is image 

224,224,3 = shape

In [39]:
def detect_face_mask(img):
    
    y_pred=(model.predict(img.reshape(1,224,224,3)>0.5).astype("int32"))
    
    return y_pred[0][0]

In [40]:
sample1=cv2.imread('sample/2 .jpg')
sample1=cv2.resize(sample1,(224,224))

In [41]:
detect_face_mask(sample1)

0

In [45]:
def draw_label(img,text,pos,bg_color):
    
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x=pos[0]+text_size[0][0]+2
    
    end_y=pos[1]+text_size[0][0]-2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)